In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [74]:
### Import Libraries 
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
import seaborn as sns
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Input
from sklearn.model_selection import train_test_split


In [75]:
### Reading csv data files using pandas dataframe 

train = pd.read_csv("/Users/ved/Desktop/Sem 7/DL Project/Dataset/train.csv", encoding = "ISO-8859-1")
test = pd.read_csv("/Users/ved/Desktop/Sem 7/DL Project/Dataset/test.csv", encoding = "ISO-8859-1")

In [76]:
### Before cleaning the dataset I would like to perform EDA(Exploratory data analysis) by performing data visualization to understand
### the distribution of different classes. I will be performing EDA on training dataset

categorywise_data = train.drop(['id', 'comment_text'], axis=1)     ### Removed unnecessary columns - id and comment_text
counts_category = []                                               ### A list that contains tuple which consists of class label and number of comments for that particular class 
categories = list(categorywise_data.columns.values)
for i in categories:
    counts_category.append((i, categorywise_data[i].sum()))
    
dataframe = pd.DataFrame(counts_category, columns=['Labels', 'number_of_comments'])   ### Dataframe made up of category and total number of comments

In [77]:
### Visualization 2

### Bar graph of Total No. of labels in a sentence against Total no. of sentences
### This visualization is helpful in identifying whether a sentence belongs to only one category or many categories

dataframe = pd.DataFrame(pd.DataFrame(train[train.columns[2:]].sum(axis=1)).reset_index()[0].value_counts())
dataframe["Total no. of sentences"]=dataframe[0]
dataframe["Total No. of labels in a sentence"]=dataframe.index

### From the below graph we can see that 1,43,346 out of 1,59,571 sentences does not have any labels(class 0).
### we can observe that a single sentence can have multiple labels. It can be a toxic sentence or it can be a toxic as well as obscene senetence.

In [78]:
### Visualization 4

### Correlation between different variables
### Correlation helps us finding relationship/dependency between different variables. 

target_data = train.drop(['id', 'comment_text'], axis=1)


### Correlation coefficient ranges from -1 to 1. Values always range between -1 imply strong negative relationship
### between variables and +1 imply a strong positive relationship between variables. 
### Values at or close to zero imply weak or no linear relationship. 
### From the correlation matrix(graph), it can be concluded that some labels are highely correlated. Those varibles are mentioned below 
### (Correlation coefficient for insult-obscene is 0.74, Correlation coefficient for toxic-obscene is 0.68 and Correlation coefficient for toxic-insult is 0.65).

In [79]:
### Splitting up the labels and data
### Training dataset is splitted into 2 parts. 1st part includes the training data(train_data) and 2nd part includes labels(train_label) 
### associated with the training data
### Test dataset is having only 1 part i.e. test data which is used to predict the labels. 

train_data = train["comment_text"]
test_data = test["comment_text"]
train_label=train[['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']]


In [80]:
import ast
from keras.preprocessing import text, sequence
from keras.preprocessing.sequence import pad_sequences
df_combined = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined.csv')
df_combined_test = pd.read_csv('/Users/ved/Desktop/Sem 7/DL Project/Tokenized_Data/df_combined_test.csv')

for i in range(len(df_combined['comment'])):
    df_combined["comment"][i] = ast.literal_eval(df_combined["comment"][i])

for i in range(len(df_combined_test['comment'])):
    df_combined_test["comment"][i] = ast.literal_eval(df_combined_test["comment"][i])

X_train1 = df_combined['comment']
X_test1 = df_combined_test['comment']

max_features = 30000
maxlen = 150
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
X_train = tokenizer.texts_to_sequences(X_train1)
X_test = tokenizer.texts_to_sequences(X_test1)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_19822/50109776.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined["comment"][i] = ast.literal_eval(df_combined["comment"][i])
/var/folders/1m/v10vjxqd235fn30s91s6nnx40000gn/T/ipykernel_19822/50109776.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined_test["comment"][i] = ast.literal_eval(df_combined_test["comment"][i])


In [81]:
train_padded = x_train
test_padded = x_test
print("Shape of training data",train_padded.shape)
print("Shape of testing data",test_padded.shape)


Shape of training data (159571, 150)
Shape of testing data (63978, 150)


In [82]:
### Creating corpus of words and coverting it into integer and then susbstituing it in sentences - prepare tokenizer
tokenizer = Tokenizer(num_words = 40000) #40000 words are used here
tokenizer.fit_on_texts(train_data)

#convert each text into array of integers with help of tokenizer.
train_final = tokenizer.texts_to_sequences(train_data)
test_final = tokenizer.texts_to_sequences(test_data)

In [83]:
### Padding - Every sentence is unequal in length. We need to have all the sentence equal in lengths
### If length of sentence is less than 150 then padding will increase sentence length to 150 by adding zeros, if its greater
### than 150 then it will reduce the length of sentence to 150 by trimming the words

train_padded =pad_sequences(train_final, maxlen=150)
test_padded =pad_sequences(test_final, maxlen=150)
print("Shape of training data",train_padded.shape)
print("Shape of testing data",test_padded.shape)


Shape of training data (159571, 150)
Shape of testing data (153164, 150)


In [93]:
from keras.models import load_model

# Load the saved model
model = load_model('LSTM.h5')

In [94]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train1) + list(X_test1))
def custom(input):
    input = tokenizer.texts_to_sequences(input)
    input = sequence.pad_sequences(input, maxlen=maxlen)
    pred = model.predict(input)
    toxicity_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    j=0
    for i in pred[0]:
        if i > 0.5:
            pred[0][j]=1
        else:
            pred[0][j]=0
        j = j+1
    for i, class_name in enumerate(toxicity_classes):
        print(f"{class_name}: {pred[0][i]}")

In [111]:
input= ["shit"]
custom(input)

1/1 [==============================] - 0s 17ms/step
toxic: 1.0
severe_toxic: 0.0
obscene: 1.0
threat: 0.0
insult: 1.0
identity_hate: 0.0
